In [1]:
import numpy as np
import pandas as pd
import re #biểu thức chính quy
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from underthesea import chunk
from underthesea import text_normalize
import swifter

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

Đọc dữ liệu

In [2]:
real_data = pd.read_csv('data/RealNewsfrom_NewsData_io_UPDATED.csv', encoding='utf-8')
fake_data = pd.read_csv('data/FakeNews.csv', encoding='utf-8')

Xác định và chuẩn bị dữ liệu

In [3]:
real_data['label'] = 0 #real
fake_data['label'] = 1 #fake

# Kết hợp hai DataFrame thành một DataFrame lớn
data = pd.concat([real_data, fake_data], ignore_index=True)
data = data.fillna('')

Gộp các trường title, content, creator lại thành 1 trường text (1 cột trong dataframe)

In [4]:
data['text'] = data['title']+ ' ' + data['content'] + ' ' + data['creator']

Mở tệp dữ liệu temp_data_noClean.csv và kiểm tra

In [5]:
data_check = pd.read_csv('data/temp_data_noClean.csv', encoding='utf-8')
print(data['text'])

0       Ông Tucker Carlson đưa ra lời cảnh báo tới ông...
1       Đức điều động binh sĩ tới Lithuania trong đợt ...
2       Thừa nhận mới nhất của ĐCSTQ về tình hình dịch...
3       Nghe tiếng khóc của đứa trẻ sơ sinh, biết trướ...
4       Con trai Trương Tấn Sang bị đẩy ra khỏi TP.HCM...
                              ...                        
1618     Ở Việt Nam bị cấm tuyệt đối không được kết hô...
1619     Người Dân trên toàn trái đất đi sơ tán vì sắp...
1620     Tất cả những tin tức trên Facebookn đều là gi...
1621     Đưa tin thất thiệt mang nhiều hệ quả rất xấu ...
1622              Đa cấp là ngành thiết yếu của Việt Nam 
Name: text, Length: 1623, dtype: object


Tối ưu dữ liệu nhập vào tránh phải chuẩn hóa các bài báo cũ:
- Bước 1: Ghép (merge) hai DataFrame data và data_check dựa trên cột 'text' với kiểu ghép là 'outer' (ghép toàn bộ).
- Bước 2: Lọc những hàng chỉ xuất hiện trong data mà không xuất hiện trong data_check.
- Bước 3: Kiểm tra missing_rows_data_in_datacheck có giá trị được truyền vào hay không:
    + Nếu không phát hiện không có giá trị nào được truyền vào thì trả về 1 dataframe rỗng chứa duy nhất các trường mặc định của data
    + Nếu có giá trị được truyền vào missing_rows_data_in_datacheck thức hiện các mục:
In ra result_value để kiểm tra

In [6]:

merged_data_datacheck = pd.merge(data, data_check, on='text', how='outer', indicator=True) # 

# Lọc các hàng chỉ xuất hiện trong data
missing_rows_data_in_datacheck = merged_data_datacheck[merged_data_datacheck['_merge'] == 'left_only']

# Kiểm tra xem missing_rows_data_in_datacheck có tồn tại hay không
if missing_rows_data_in_datacheck.empty:
    result_value = pd.read_csv('data/temp.csv')
else:
    # Nếu có, thực hiện các bước tiếp theo
    data_check = data.copy()                                        #-> copy toàn bộ dữ liệu từ dataframe của data để update data_check (thêm những bài báo mới)
    data_check.to_csv('data/temp_data_noClean.csv', index=False)    #-> đẩy data_check thành 1 file csv để thay thế file cũ (chứa những bài báo cũ lần đã cập nhật)
    temp = missing_rows_data_in_datacheck.copy()                    #-> copy những bài báo data có mà data_check không có vào biến temp
    temp.rename(columns={'label_x': 'label'}, inplace=True)         #-> thay đổi tên trường trong dataframe temp cho phù hợp với dữ liệu đúng
    selected_columns = ['text', 'label']                            #-> tạo một biến mang 2 trường là text và label
    temp_selected = temp[selected_columns]                          #-> tạo 1 dataframe temp_selected chỉ mang 2 trường là text và label của temp
    result_value = temp_selected                                    #-> trả về giá trị cuối là result_value

# In kết quả hoặc sử dụng result_value theo nhu cầu
print(result_value)


                                                  text  label
843  Người dân, du khách về Bà Rịa - Vũng Tàu xem c...      0
844  Thu ngân sách TP.HCM năm 2023 mới đạt 95,07% T...      0
845  Ô tô tồn kho từ 2022 miệt mài xả hàng, vừa bán...      0
846  Vì sao Lưu Bang phải đút lót, gả con gái cho T...      0
847  Khởi tố, khám nhà cựu Bí thư Tỉnh ủy Thanh Hóa...      0
848  Ukraine bắn rụng 27 UAV và gần 90 tên lửa tron...      0
849  Loại quả "trái cây của người bệnh tiểu đường" ...      0
850  Cần Thơ: Bắt nguyên Phó Chủ tịch UBND huyện Vĩ...      0
851  Ngân hàng Nhà nước tiết lộ kế hoạch can thiệp,...      0
852  Đông y chữa tiểu tiện khó Bàng quang sở dĩ thả...      0
853  Vô địch tại Việt Nam, đội bóng Thái Lan hành đ...      0
854  Honda HR-V 2023 ‘full option’ giảm giá sốc trê...      0
855  'Nỗi ác mộng' của Messi chính thức giải nghệ T...      0
856  CHÍNH THỨC: Real Madrid gia hạn hợp đồng với C...      0
857  Ronaldo lại 'gây bão' với bình luận về Haaland...      0
858  Tiế

Chuẩn hóa, Phân tích từ

In [7]:
def word(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    chunks = chunk(text)
    filtered_chunks = [chunk[0] for chunk in chunks if chunk[2] != 'O']
    result_sentence = ' '.join(filtered_chunks)
    return result_sentence

result_value.loc[:, 'text'] = result_value['text'].swifter.apply(word).copy()

Pandas Apply:   0%|          | 0/39 [00:00<?, ?it/s]

In [8]:
data_have_clean = pd.read_csv('data/temp_data_noClean.csv', encoding='utf-8')

In [9]:
data = pd.concat([data_have_clean, result_value[['text', 'label']]], ignore_index=True)
data = data.fillna('')
print(data['text'])

0       Ông Tucker Carlson đưa ra lời cảnh báo tới ông...
1       Đức điều động binh sĩ tới Lithuania trong đợt ...
2       Thừa nhận mới nhất của ĐCSTQ về tình hình dịch...
3       Nghe tiếng khóc của đứa trẻ sơ sinh, biết trướ...
4       Con trai Trương Tấn Sang bị đẩy ra khỏi TP.HCM...
                              ...                        
1657    tranh bích họa việt nam tươi đẹp mang diện mạo...
1658    5 bộ phim hàn quốc có rating cao 2023 phải xem...
1659    loạt nam thần tượng kpop hàng đầu trong danh s...
1660    phát hiện chồng chụp ảnh cưới với người phụ nữ...
1661    cựu sao al nassr ghi bàn tốt hơn ronaldo được ...
Name: text, Length: 1662, dtype: object


In [10]:
data.isnull().sum()

article_id     0
title          0
link           0
creator        0
description    0
content        0
source_id      0
category       0
label          0
text           0
dtype: int64

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    data['text'],
    data['label'],
    test_size=0.2,
    random_state=42
    )

In [12]:
vectorized = TfidfVectorizer()
param_grid = {'kernel': ['linear', 'rbf'], 'C': [0.1, 1, 10], 'gamma': ['scale', 'auto']}
X_train_tfidf = vectorized.fit_transform(X_train)
X_test_tfidf = vectorized.transform(X_test)

In [13]:
svm_model = SVC()
grid_search = GridSearchCV(svm_model, param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train_tfidf, y_train)

GridSearchCV(cv=5, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10], 'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'rbf']})

In [14]:
print('Best hyperparameters:', grid_search.best_params_)


Best hyperparameters: {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}


In [15]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_tfidf)


In [16]:
data = data.loc[y_test.index]
data.loc[y_test.values != y_pred, 'label'] = 2


In [17]:
y_pred

array([1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0,

In [18]:
import pickle
with open('temp/vectorized.pkl', 'wb') as f:
    pickle.dump(vectorized, f)  
with open('temp/svm_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)
with open('temp/word_clean.pkl', 'wb') as f:
    pickle.dump(word, f)

In [19]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Độ chính xác của mô hình SVM: {accuracy}')

Độ chính xác của mô hình SVM: 0.9369369369369369


In [20]:
print(len(y_test), len(y_pred))

333 333


In [21]:
print(f"Số hàng trong X_train: {len(X_train)}")
print(f"Số phần tử trong y_train: {len(y_train)}")
print(f"Số hàng trong X_test: {len(X_test)}")
print(f"Số phần tử trong y_test: {len(y_test)}")

Số hàng trong X_train: 1329
Số phần tử trong y_train: 1329
Số hàng trong X_test: 333
Số phần tử trong y_test: 333


In [22]:
from sklearn.metrics import confusion_matrix, classification_report

# Dự đoán trên tập kiểm tra
y_pred = best_model.predict(X_test_tfidf)

# Tính ma trận nhầm lẫn
conf_matrix = confusion_matrix(y_test, y_pred)

# In ma trận nhầm lẫn
print("Confusion Matrix:")
print(conf_matrix)

# In báo cáo đánh giá
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Số lượng tin thật và tin giả được dự đoán đúng
true_real_news = conf_matrix[1, 1]
true_fake_news = conf_matrix[0, 0]

print(f"\nSố lượng tin thật được dự đoán đúng: {true_real_news}")
print(f"Số lượng tin giả được dự đoán đúng: {true_fake_news}")


Confusion Matrix:
[[176  10]
 [ 11 136]]

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.95      0.94       186
           1       0.93      0.93      0.93       147

    accuracy                           0.94       333
   macro avg       0.94      0.94      0.94       333
weighted avg       0.94      0.94      0.94       333


Số lượng tin thật được dự đoán đúng: 136
Số lượng tin giả được dự đoán đúng: 176


In [23]:
# Giả sử 'content' là tên của cột chứa nội dung bài báo trong DataFrame của bạn
# có bao nhiêu bài báo chưa phân biệt được
print(f'Số lượng bài báo chưa thể phân biệt được: {len(data[data["label"] == 2])}')
#Số lượng bài báo đã phân biệt được
print(f'Số lượng bài báo đã phân biệt được: {len(data[data["label"] != 2])}')

print('\nDanh sách bài báo chưa thể phân biệt được cùng với nội dung:')
for i in range(len(y_test)):
    if y_test.values[i] != y_pred[i]:
        index = i
        content_of_article = data['text'].iloc[index]
        actual_label = y_test.values[index]
        predicted_label = y_pred[index]

        print(f'\nIndex: {index}, Thực tế: {actual_label}, Dự đoán: {predicted_label}')
        print(f'Nội dung bài báo:\n{content_of_article}\n')


Số lượng bài báo chưa thể phân biệt được: 21
Số lượng bài báo đã phân biệt được: 312

Danh sách bài báo chưa thể phân biệt được cùng với nội dung:

Index: 2, Thực tế: 0, Dự đoán: 1
Nội dung bài báo:
Nên học Công nghệ thông tin hay Khoa học máy tính? Em cũng dành thời gian đọc tài liệu trên mạng nhưng thấy thông tin mâu thuẫn nhau. Nơi thì bảo học Công nghệ thông tin, có trang lại nói cần học Khoa học máy tính. Vậy em nên chọn ngành nào để sau này có thể làm vị trí kiểm thử? Em rất mong các anh, chị đang làm về kiểm thử phần mềm cho em lời khuyên, chia sẻ quá trình mọi người đến với ngành này. Hà 


Index: 30, Thực tế: 1, Dự đoán: 0
Nội dung bài báo:
 Khi ông Kim Jong-il, cha của ông Kim Jong-un, qua đời năm 2011, các quan chức tình báo nước ngoài không hay biết cho đến khi tin tức được công bố hai ngày sau đó trên truyền hình Triều Tiên.

Có lẽ nào???

 


Index: 39, Thực tế: 1, Dự đoán: 0
Nội dung bài báo:
 Clip camera quay lại cảnh nữ nhân viên xe buýt bị đâ.m chế.t ở Củ Chi
 -------